<a href="https://colab.research.google.com/github/liuy01510/portfolio/blob/master/Python/Project-Euler/Project_Euler_Problem_70_PF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
- Find $n$ where $\phi(n)$ is a permutation of $n$.
    - Condition $\rightarrow$ minimize $\frac{n}{\phi(n)}$

## Solution derivation
- In order to minimize $\frac{n}{\phi(n)}$, $n$ must be minimized while $\phi(n)$ must be maximized.
    - Recall that $\frac{n}{\phi(n)}=\frac{1}{\prod \left(1-\frac{1}{p_i} \right)}$
    - Therefore, to minimize $\frac{n}{\phi(n)}$ by maximizing $\phi(n)$, the number of product terms in the denominator must be minimized since each product term < 1.
    - Also, $p_i$ should be maximized.
- $\therefore$ Starting with the largest prime factor, perform the computation of $\phi(n)$ and permute the result until the condition is met.

# Importing modules

In [29]:
# GENERAL MODULES #
import collections as coll
import requests
from numba import jit
from math import sqrt, ceil
from time import time
import numpy as np

## Importing functions written in Problem 69

In [30]:
url='https://raw.githubusercontent.com/liuy01510/portfolio/master/Python/Project-Euler/Project_Euler_Problem_69_(PF).ipynb'
def Import_Notebook(url,file_name):
    """
    Used for importing jupyter notebooks from Github. Currently only supports public notebooks.

    Args:

    - url:str = Url to the jupyter notebook to be imported.

    - file_name:str = Name of the file for the jupyter notebook to be saved as.
    """
    #GETTING THE FILE FROM GITHUB#
    url=url
    req=requests.get(url)
    res=req.content
    with open(f'/content/{file_name}.ipynb','wb') as f:
        f.write(res)

    #CONVERTING TO .PY FILETYPE AND IMPORTING IT#
    !jupyter nbconvert --to script Problem_69.ipynb # convert to .txt file
    try:
        !mv Problem_69.txt Problem_69.py # convert from .txt to .py extension
    except:
        pass

#IMPORTING THE MODULES REQUIRED#
try:
    from Problem_69 import Prime_Sieve
except:
    Import_Notebook(url,'Problem_69')
    from Problem_69 import Prime_Sieve

# Defining Functions
- The Prime_Sieve function imported from Problem 69 is too slow to generate all possible primes until $10^7$.
    - Therefore, a new prime number generator function has to be written that can handle the generation of primes in descending order from a given upper limit.

In [31]:
@jit(nopython=True)
def Prime_Number_Generator(ul=None,order='asc'):
    """
    Generator function for prime numbers.

    Args:

    - ul:int = Upper limit of the generator function. 
    Optional if order=='asc', required if order=='desc'

    - order:'asc','desc' = Determines the order of generation of the prime numbers.

    Returns:

    - result:int = Prime number.

    Performance:

    - Around 1.65s for ul==10**15 (Colab). Not recommended for use above this limit.
    """

    def Prime_Test(num):
        if (num==2) or (num==3):
            return True
        if num%2==0: # even number
            return False
        if num%3==0:
            return False
        
        # prime test divisors
        k=1
        div=[]
        while (6*k)-1<sqrt(num):
            a,b=(6*k)+1,(6*k)-1
            div.append(a)
            div.append(b)
            k+=1

        # prime test
        for d in div:
            if num%d==0:
                return False # divisible by a prime
        return True # non-divisible by any primes

    # Adjusting to odd number
    if ul%2==0:
        ul=ul-1 # change to odd number
    
    if order=='asc':
        i=3
        yield i-1 # yields 2
        while True:
            if Prime_Test(i):
                yield i
            i+=2
            if (ul!=None) and (i>ul):
                raise StopIteration

    if order=='desc':
        i=ul
        while True:
            if Prime_Test(i):
                yield i
            i+=-2
            if i==3:
                yield i # yields 3
                yield i-1 # yields 2
                raise StopIteration

In [32]:
# Prime_Factors function - Return the prime factors of a given number
def Prime_Factors(num):
    """
    Gets the prime factors for a non-prime number.

    Args:

    - num:int = Non-prime number to find the prime factors for.

    Returns:

    - res:list = List of prime factors.
    """
    n=num
    primes=Prime_Number_Generator(num)
    p=next(primes)
    res=[]
    while n!=1:
        if n%p==0:
            res.append(p)
            n=int(n/p)
            continue
        else:
            p=next(primes)
    return res

def Euler_Toitent(num,primes=None):
    """
    Gets the Euler Toitent value for a specific number.

    Args:

    - num:int = Number to find the Euler Toitent value for.

    Returns:

    - n:int = Euler Toitent value.
    """
    n=num
    if primes==None:
        primes=set(Prime_Factors(num))
    else:
        primes=set(primes)
    for p in primes:
        n*=(1-(1/p))
    return int(n)

# Permutation check function

In [33]:
def Permutations_Check(str1,str2):
    """
    Checks if 2 strings are permutations of each other.

    Args:

    - str1:str = 1st string to check.

    - str2:str = 2nd string to be counter checked against the 1st string.

    Returns:

    - result:bool = True if str 1 and str2 are permutations of each other.
    """
    a,b=coll.Counter(str1),coll.Counter(str2)
    return True if a==b else False

# Solution

In [34]:
def Main():
    #INITIALIZE VARIABLES#
    lowPrimes=Prime_Number_Generator(ul=10**7,order='asc')
    lPrime=next(lowPrimes)
    results=[]

    #INITIALIZE LOOP#
    while True:
        uPrimeLim=ceil((10**7)/lPrime)
        for uPrime in Prime_Number_Generator(ul=uPrimeLim,order='desc'):
            if uPrime<lPrime:
                break # break the for loop
            n=uPrime*lPrime
            eT=Euler_Toitent(n,[lPrime,uPrime])
            if Permutations_Check(str(n),str(eT)):
                results.append((n,eT,n/eT))
        if lPrime>sqrt(10**7):
            break
        lPrime=next(lowPrimes)
    
    #RESULT#
    result=sorted(results,key=lambda x:x[2],reverse=False)[0]
    n,eT,ratio=result
    print(f"The number {n} produces a permuted Euler Toitent value of {eT}, with the smallest ratio of {ratio}.")

start=time()
Main()
end=time()
print(f"The total time taken to solve Problem 70 is {end-start} seconds.")

The number 8319823 produces a permuted Euler Toitent value of 8313928, with the smallest ratio of 1.0007090511248113.
The total time taken to solve Problem 70 is 41.73289728164673 seconds.


# Discussion
- This problem was quite challenging, and required siginificantly more research than the other problems.
- Furthermore, there seems to be a potential problem with the phrasing of the question that causes it to be unsolvable if no assumptions were made.
    - In this solution method, the algorithm makes the assumption that the lowest $\frac{n}{\phi(n)}$ comes from a pair of prime factors, and not a group of $\geq 3$ prime factors.
    - This assumption is required in order to avoid searching through all possible group sizes of prime factors, which would be equivalent to computing the prime factors of each non-prime digit individually (infeasible).
    - However, just because a solution is found from a pair of prime factors, it does not guarantee that this is the optimal solution since there may be other groups of prime factors that may have a smaller $\frac{n}{\phi(n)}$ ratio than what was produced from the prime pair.
- However, since again it would be infeasible to compute all the prime factors for $1 \leq n \leq 10^7$, combined with the fact that a increase in prime factors is likely to cause the ratio to be deviate from the minimum, this solution is accepted as the most likely solution.